In [ ]:
#####################
# AUTO LOGIN-1 of 1 #
#####################

import login
from login import *
import pandas as pd

In [ ]:
!runipy test123.ipynb

In [ ]:
while True:
    !runipy test123.ipynb

In [ ]:
#####################
#    PARAMENTERS    #
#####################

this_month_token = 690691
next_month_token = 780803

this_month_symbol = 'USDINR20OCTFUT'
next_month_symbol = 'USDINR20NOVFUT'

# buysell_diff = 0.2100                                 # 0.2100 per day
# squareup_diff = 0.1950                                # 0.1950 per day

fullquant = 400

order_type = 'MIS'

In [ ]:
difference_ip = 0.0100

most_repeated_diff = 0.2000

upper_diff = most_repeated_diff + (difference_ip/2)
lower_diff = most_repeated_diff - (difference_ip/2)

print('Upper Diff : '+str(round(upper_diff,4)))
print('Lower Diff : '+str(round(lower_diff,4)))

In [ ]:
symbol_ip = this_month_symbol
symbol_ip2 = next_month_symbol

inst_token = this_month_token
inst_token2 = next_month_token

myquantity = 0
myquantity2 = 0

def getquant():
    global myquantity,myquantity2
    global order_type
    global symbol_ip
    allpos = kite.positions()['net']
    for i in range(len(allpos)):
        if(allpos[i]['tradingsymbol']==symbol_ip and  allpos[i]['product']== order_type):
            myquantity = allpos[i]['quantity']
            print('My Quantity 1 : ' + str(allpos[i]['quantity']))
        if(allpos[i]['tradingsymbol']==symbol_ip2 and  allpos[i]['product']== order_type):
            myquantity2 = allpos[i]['quantity']
            print('My Quantity 2 : ' + str(allpos[i]['quantity']))

getquant()

def placeneworder(quantdiff,symbol_ipp):
    global stopbuy
    global stopsell
    global order_type
#     global symbol_ip,symbol_ip2
    if(quantdiff>0):
        try:
                order_id= kite.place_order(tradingsymbol=symbol_ipp,
                            exchange=kite.EXCHANGE_CDS,
                            transaction_type=kite.TRANSACTION_TYPE_BUY,
                            quantity=abs(quantdiff),
                            order_type=kite.ORDER_TYPE_MARKET,
                            variety = kite.VARIETY_REGULAR,
                            product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
                getquant()
        except Exception as e:
            print(e)
            getquant()
    if(quantdiff<0):
        try:
                order_id= kite.place_order(tradingsymbol=symbol_ipp,
                            exchange=kite.EXCHANGE_CDS,
                            transaction_type=kite.TRANSACTION_TYPE_SELL,
                            quantity=abs(quantdiff),
                            order_type=kite.ORDER_TYPE_MARKET,
                            variety = kite.VARIETY_REGULAR,
                            product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
                getquant()
        except Exception as e:
            print(e)
            getquant()


def ckqnt (orderquant):
    global myquantity,myquantity2,symbol_ip,symbol_ip2
    quantdiff = orderquant - myquantity
    quantdiff2 = -orderquant - myquantity2
    
    placeneworder(quantdiff,symbol_ip)
    placeneworder(quantdiff2,symbol_ip2)
    return myquantity

In [ ]:
#################
# START TRADING #
#################

import logging
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)
# kws = KiteTicker("w19o0chuo929jxkp", "eA5B5OJQNOtZ0OYihkBmYw7Ke3B9pmCC")

def on_ticks(ws, ticks):
    #print(type(ticks[0]['last_price']))
    print('==========================')
    print( 'LTP1 : ' + str(ticks[0]['last_price']))
    print( 'LTP2 : ' + str(ticks[1]['last_price']))
    ltp_diffs = ticks[1]['last_price'] - ticks[0]['last_price']
    print( 'Difference : ' + str(ltp_diffs))
    if(ltp_diffs==buysell_diff):
        ckqnt(int(fullquant/2))
    if(ltp_diffs==squareup_diff):
        ckqnt(0)
#     print('==========================')
#     print()

def on_connect(ws, response):
    global inst_token,inst_token2
    ws.subscribe([inst_token,inst_token2])
    ws.set_mode(ws.MODE_FULL, [inst_token,inst_token2])

def on_close(ws, code, reason):
    ws.stop()
    
def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))

def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))

#def on_order_update(ws, data):
#    print("order update: ", data)

kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
#kws.on_order_update = on_order_update

kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

kws.connect()

In [ ]:
################################
#      TECHNICAL  AANALYSIS    #
################################

from datetime import timedelta
import datetime
pd.set_option('plotting.backend', 'pandas_bokeh')
import pandas_bokeh
pandas_bokeh.output_notebook()

old_lst=[]
old_lst2=[]

interval='minute'
todaydt=datetime.date.today()
hud_ago=todaydt-timedelta(days=59)
to_date=datetime.date.isoformat(todaydt)
from_date=datetime.date.isoformat(hud_ago)
inst_token = 690691
inst_token2 = 780803
for i2 in range(50):
    
    new_lst = kite.historical_data(inst_token, from_date, to_date, interval,continuous=False)
    old_lst = new_lst + old_lst
    
    new_lst2 = kite.historical_data(inst_token2, from_date, to_date, interval,continuous=False)
    old_lst2 = new_lst2 + old_lst2
    
    todaydt=todaydt-timedelta(days=60)
    hud_ago=hud_ago-timedelta(days=60)
    to_date=datetime.date.isoformat(todaydt)
    from_date=datetime.date.isoformat(hud_ago)
    print(len(old_lst))
    
mydf1 = pd.DataFrame(old_lst)
mydf2 = pd.DataFrame(old_lst2)

In [ ]:
day_ago = 2

badbaki = 480*(day_ago - 1)
oneday1 = mydf1['close'][107292-badbaki:107772-badbaki].to_numpy()
oneday1.shape
# mydf1[107292-badbaki:107772-badbaki].groupby('close').count()

In [ ]:
oneday2= mydf2['close'][98984-badbaki:99464-badbaki].to_numpy()
oneday2.shape
# mydf2[98984-badbaki:99464-badbaki].groupby('close').count()

In [ ]:
mydf2[98984-badbaki:99464-badbaki]

In [ ]:
mydiff = oneday2 - oneday1
pd.DataFrame(mydiff).plot.hist(bins=100, alpha=0.8)
# 1900 -> 2
# 1925 -> 8
# 1950 -> 35
# 1975 -> 90
# 2000 -> 152
# 2025 -> 116
# 2050 -> 66
# 2075 -> 11

In [ ]:
pd.DataFrame(mydiff).describe()

In [ ]:
round(pd.DataFrame(mydiff).describe()[:][5:6].to_numpy()[0][0],4)

In [ ]:
np1 = mydf1[92891:].to_numpy()
np1.shape

In [ ]:
np2 = mydf2[85063:99463]
np2.shape

In [ ]:
newdf2 = mydf2[293-30:293]
newdf1 = mydf1[292-30:292]

newdf1np = newdf1.to_numpy()
newdf2np = newdf2.to_numpy()

newdf1np[:,4:5]

difff = newdf2np[:,4:5] - newdf1np[:,4:5] 
difff

In [ ]:
diffpd = newdf2['close'] - newdf1['close']
diffpd.plot.hist(bins=100, alpha=0.8)  

In [ ]:
difffpd = pd.DataFrame(difff)
difffpd.plot.hist(bins=1, alpha=0.8)  

In [ ]:
np1 = mydf1['close'][:].to_numpy()
np2 = mydf2['close'][1:].to_numpy()
dif12 = np2 - np1
difpd = pd.DataFrame(dif12)
difpd[80:].plot.hist(bins=100, alpha=0.8)  

In [ ]:
# NOTE
# SET UPPER circuit lower circuit limit
# Time 16:40
# Rejection handling